Step 1: Import Libraries and Set Up Embeddings

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import Tool
from langchain.schema import AgentAction, AgentFinish
import os
import os

# Set the API key directly
os.environ["OPENAI_API_KEY"] = 



# Load pre-trained embeddings (e.g., OpenAI or other provider)
embeddings = OpenAIEmbeddings()

# Sample trading strategies embeddings
trading_strategies = [
    {"strategy_name": "Mean Reversion", "description": "Buy when the price deviates significantly below its average."},
    {"strategy_name": "Momentum Trading", "description": "Buy assets showing upward price momentum."},
    {"strategy_name": "Arbitrage", "description": "Exploit price differences between markets or instruments."},
    # Add more strategies as needed
]

# Convert descriptions into embeddings
texts = [s["description"] for s in trading_strategies]
faiss_index = FAISS.from_texts(texts, embeddings)


Step 2: Define a Custom Tool for Strategy Creation

In [13]:
def generate_trading_strategy(input_data: str, similar_strategies: list) -> str:
    """
    Generate a new trading strategy based on user input and similar strategies.
    """
    strategy_base = f"User Input: {input_data}\n\nSimilar Strategies:\n"
    for idx, strategy in enumerate(similar_strategies):
        strategy_base += f"{idx + 1}. {strategy['strategy_name']}: {strategy['description']}\n"
    
    strategy_base += "\nNew Strategy:\n"
    strategy_base += "Using the above information, synthesize a unique and practical trading strategy."
    
    return strategy_base


def strategy_creation_tool(input_data: dict) -> str:
    """
    Create a trading strategy using user input and similar strategies.
    Args:
        input_data (dict): Dictionary with "user_input" (str) and "k" (int, optional).
    Returns:
        str: A synthesized trading strategy.
    """
    # Validate input data
    if not isinstance(input_data, dict):
        raise ValueError("Input data must be a dictionary.")
    
    user_input = input_data.get("user_input", "")
    k = input_data.get("k", 3)  # Number of similar strategies to consider
    
    if not user_input:
        raise ValueError("User input is required.")
    
    # Find similar strategies
    try:
        docs = faiss_index.similarity_search(user_input, k=k)
        similar_strategies = [
            {
                "strategy_name": doc.metadata.get("strategy_name", "Unnamed Strategy"),
                "description": doc.page_content,
            }
            for doc in docs
        ]
    except Exception as e:
        return f"Error in similarity search: {e}"
    
    # Generate the new trading strategy
    return generate_trading_strategy(user_input, similar_strategies)


Step 3: Integrate the Tool into a LangChain Agent

In [18]:
from langchain.agents import Tool, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

# Define the tool for strategy creation
tools = [
    Tool(
        name="Strategy Creation Tool",
        func=strategy_creation_tool,
        description="Creates a new trading strategy based on user input and embeddings of existing strategies.",
    )
]

# Initialize the OpenAI language model
llm = OpenAI(temperature=0.7)  # Using the updated langchain_openai package

# Define a custom prompt for the agent
prompt = PromptTemplate(
    input_variables=["tool_input", "agent_scratchpad", "tools", "tool_names"],
    template="""
    You are a highly intelligent assistant specialized in creating trading strategies.
    Use the tools provided to generate actionable insights.

    Available Tools:
    {tools}

    Tool Names:
    {tool_names}

    User Input:
    {tool_input}

    {agent_scratchpad}
    """
)

# Create the agent
agent_executor = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

# Define the agent's interface
def run_agent(user_input: str, k: int = 3):
    tool_input = {"user_input": user_input, "k": k}
    response = agent_executor.run(tool_input)
    return response


Step 4: Example Usage

In [21]:
if __name__ == "__main__":
    # Example user input
    user_input = "I want a trading strategy that minimizes risk and leverages arbitrage opportunities."
    k = 3  # Number of similar strategies to consider

    # Tool input preparation
    tool_input = {"user_input": user_input, "k": k}

    # Run the agent
    try:
        response = agent_executor(tool_input)  # Use agent_executor as callable
        print("\nGenerated Trading Strategy:")
        print(response)
    except Exception as e:
        print(f"Error while running the agent: {e}")


Error while running the agent: 'RunnableSequence' object is not callable
